Packages

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import random

Calculations

In [10]:
uCi = 3.7 * (10**10) * (10**-6)
year = 3.154*(10**7)

activity_0 = 57 * uCi
u_activity_0 = 0.5 * uCi
half_life = 87.7 * year
u_half_life = 0.05 * year
age = 54 * year
u_age = (1/24)*year

activity_expected = activity_0 * np.exp(-np.log(2)*age/half_life)
u_activity_expected = activity_expected * np.sqrt((u_activity_0/activity_0)**2 + (u_age/age)**2 + (u_half_life/half_life)**2)

inch = 2.54 #cm
amu = 1.6605390689*(10**-24)

density = 19.283 #g/cm3
mass_Au = 196.96657 * amu #g

e_0 = 55.26349406 * (10**6) * 10000 #e2 MeV-1 cm-1
k = 1/(4*pi*e_0)
q = 2 #e
Q = 79 #e
E = 5 #MeV
u_E = 0.5


E_squared = E**2
u_E_squared = E_squared*2*u_E/E

z_expected = 0.0002 * inch
u_z_expected = 0.00005 * inch

n_tar = density*z_expected/mass_Au
u_n_tar = density*u_z_expected/mass_Au

A_expected = n_tar*((k*q*Q/(4))**2)/E_squared
u_A_expected = A_expected* np.sqrt((u_n_tar/n_tar)**2 + (u_E_squared/E_squared)**2)

print(A_expected)
print(u_A_expected)

3.875738508679358e-05
1.240841759143832e-05


Physics Code

In [11]:
xs_min = 1.10 + 0.6 #cm, min distance + thumbscrew thickness
xd_min = 3.66 #cm
z_Au = 0.0005 #cm, or 0.0002 in
d_Au = 0.1 #cm, or 0.040 in
d_Pu = 1.3 #cm
d_det = 0.7 #cm
pi = np.pi


c = 3 * 10**10 #cm/s
m_alpha = (3.727379 * 10**3)/(c*c) #MeV/c2
E_alpha = 5 #MeV
dE_alpha = 1 #MeV, energy lost in gold

# returns speed of particle in cm/s from kinetic energy E (MeV) and mass (MeV/c2)
def speed(E, m):
    return np.sqrt(2*E/m)

# Alpha partilce object. Contains kinematic variables needed for simulation
class Alpha:
    def __init__(self, t, x, v):
        self.t = t
        self.x = x
        self.v = v

    # returns radial position
    def r(self):
        return np.sqrt(self.x[0]**2 + self.x[1]**2)
    
    # return spherical components of velocity: speed, theta, phi
    def v_sph(self):
        s = np.linalg.norm(self.v)
        theta = np.arccos(self.v[2]/s)
        phi = np.arccos(self.v[0]/(s*np.sin(theta)))
        return np.array([s, theta, phi])

    # given z position and diameter of target (centered at r = 0), returns True and updates position+time if alpha hits target 
    def hit(z_targ, d_targ):
        if self.v[2] <= 0:
            return False
        delt = (z_targ-self.x[2])/self.v[2]
        self.t += delt
        self.x += delt*self.v
        return self.r() < d_targ/2
    
    # given change in speed and scattering angle, updates velocity
    def deflect(ds, dtheta, dphi):
        v_sph = self.v_sph() + np.array([ds, dtheta, dphi])
        self.v = v_sph[0] * np.array([np.sin(v_sph[1])*np.cos(v_sph[2]), np.sin(v_sph[1])*np.sin(v_sph[2]), np.cos(v_sph[1])])


# Creates alpah particle with random position (uniform in area of disc-shaped source) and direction (uniform in +z direction)
def generate(t):
    r = (d_Pu/2) * np.sqrt(random.random())
    x_phi = (2*pi) * random.random()
    v_theta = np.arccos(random.random())
    v_phi = (2*pi) * random.random()
    x = np.array([r*np.cos(x_phi), r*np.sin(x_phi), 0])
    s = speed(E_alpha, m_alpha)
    v = s * np.array([np.sin(v_theta)*np.cos(v_phi), np.sin(v_theta)*np.sin(v_phi), np.cos(v_theta)])
    return Alpha(t, x, v)

# checks if alphas hit foil, scatters alphas uniformly in solid angle, then checks if alphas hit detector
def uniform_scatter(a, xs, xd):
    dtheta = None
    hit_detector = False
    hit_foil = a.hit(xs_min + xs, d_Au)
    if hit_foil:
        dtheta =  np.arccos(random.uniform(-1,1))
        dphi = (2*pi) * random.random()
        ds = speed(E_alpha, m_alpha) - speed(E_alpha-dE_alpha, m_alpha)
        a.deflect(ds, dtheta, dphi)
        hit_detector = a.hit(xs_min + xs + xd_min + xd, d_det)
    return [hit_foil, hit_detector, dtheta, a.t]

# checks if alphas hit foil, scatters alphas with rutherford distribution (given normalization), then checks if alphas hit detectors
def rutherford_scatter(a, xs, xd, A):
    dtheta = None
    hit_detector = False
    hit_foil = a.hit(xs_min + xs, d_Au)
    if hit_foil:
        # mininum scattering angle (min_theta) determiend by normalization of differential cross section from min_theta to pi, with factor of A
        min_f = -1-2*A #f(min_theta)
        max_f = -2*A #f(pi)
        f = random.uniform(min_f, max_f) #flat distribution in f(theta) = -2csc^2(x/2), integral of (sinx)/(sin^4(x/2))
        dtheta =  2*np.arccsc(np.sqrt(-f/(2*A))) # generates random theta between min_theta and pi, weighted by (sinx)/(sin^4(x/2)) 
        dphi = random.uniform(0, 2*pi)
        ds = speed(E_alpha, m_alpha) - speed(E_alpha-dE_alpha, m_alpha)
        a.deflect(ds, dtheta, dphi)
        hit_detector = a.hit(xs_min + xs + xd_min + xd, d_det)
    return [hit_foil, hit_detector, dtheta, a.t]

# checks if alphas hit foil, scatters alphas with rutherford distribution modified by incidence angle, then checks if alphas hit detectors
def rutherford_scatter2(a, xs, xd, A):
    dtheta = None
    hit_detector = False
    hit_foil = a.hit(xs_min + xs, d_Au)
    if hit_foil:
        A = A/np.cos(a.v_sph()[1]) # distance through gold = target thickness weighted by secant of angle of incidence
        # mininum scattering angle (min_theta) determiend by normalization of differential cross section from min_theta to pi, with factor of A
        min_f = -1-2*A #f(min_theta)
        max_f = -2*A #f(pi)
        f = random.uniform(min_f, max_f) #flat distribution in f(theta) = -2csc^2(x/2), integral of (sinx)/(sin^4(x/2))
        dtheta =  2*np.arccsc(np.sqrt(-f/(2*A))) # generates random theta between min_theta and pi, weighted by (sinx)/(sin^4(x/2)) 
        dphi = random.uniform(0, 2*pi)
        ds = speed(E_alpha, m_alpha) - speed(E_alpha-dE_alpha, m_alpha)
        a.deflect(ds, dtheta, dphi)
        hit_detector = a.hit(xs_min + xs + xd_min + xd, d_det)
    return [hit_foil, hit_detector, dtheta, a.t]

Physics Tests

Simulations

In [ ]:
# simulates generation, scattering, and detection of N alpha particles given xs, xd
# models generation of alphas at random time every 1/N_dot second (actiivty = N_dot Hz)
# models uniform scattering, then weights number of scattered particles by rutherford distribution 
def simulate1(xs, xd, N, N_dot):
    t = 0
    T = 0
    hit_angles = []
    miss_angles = []
    N_incident = 0
    N_scattered = 0
    N_detected = 0
    for n in range(N):
        T = max(T, t)
        a = generate(t + random.random()/N_dot)
        result = uniform_scatter(a, xs, xd)
        if result[0]:
            N_incident += 1
            N_scattered += 1/((np.sin(result[2]/2))**4)
            miss_angles.append(result[2])
        if result[1]:
            N_detected += 1/((np.sin(result[2]/2))**4)
            hit_angles.append(result[2])
            T = max(T, result[3])
        t += 1/N_dot
    return [N_incident, N_scattered, N_detected, miss_angles, hit_angles, T]

# simulates generation, scattering, and detection of N alpha particles given xs, xd, and rutherford normalization A
# models generation of alphas at random time every 1/N_dot seconds (activity = N_dot Hz)
# models rutherford scattering
def simulate2(xs, xd, A, N, N_dot):
    t = 0
    T = 0
    hit_angles = []
    miss_angles = []
    N_incident = 0
    N_scattered = 0
    N_detected = 0
    for n in range(N):
        T = max(T, t)
        a = generate(t + random.random()/N_dot)
        result = rutherford_scatter(a, xs, xd, A)
        if result[0]:
            N_incident += 1
            N_scattered += 1
            miss_angles.append(result[2])
        if result[1]:
            N_detected += 1
            hit_angles.append(result[2])
            T = max(T, result[3])
        t += 1/N_dot
    return [N_incident, N_scattered, N_detected, miss_angles, hit_angles, T]

# simulates generation, scattering, and detection of N alpha particles given xs, xd, and rutherford normalization A
# models generation of alphas at random time every 1/N_dot seconds (activity = N_dot Hz)
# models rutherford scattering with distribution modified by angle of incidence
def simulate3(xs, xd, A, N, N_dot):
    t = 0
    T = 0
    hit_angles = []
    miss_angles = []
    N_incident = 0
    N_scattered = 0
    N_detected = 0
    for n in range(N):
        T = max(T, t)
        a = generate(t + random.random()/N_dot)
        result = rutherford_scatter2(a, xs, xd, A)
        if result[0]:
            N_incident += 1
            N_scattered += 1
            miss_angles.append(result[2])
        if result[1]:
            N_detected += 1
            hit_angles.append(result[2])
            T = max(T, result[3])
        t += 1/N_dot
    return [N_incident, N_scattered, N_detected, miss_angles, hit_angles, T]

# performs simulation 1 (uniform) for range of xs, xd
def experiment1(xs_list, xd_list, N):
    data = []
    for xs in xs_list:
        for xd in xd_list:
            data.append([xs, xd] + simulate1(xs, xd, N, 1))
    return data

# performs simulation2 (rutherford) for range of xs, xd, A
def experiment2(xs_list, xd_list, A_list, N):
    data = []
    for xs in xs_list:
        for xd in xd_list:
            for A in A_list:
                data.append([xs, xd, A] + simulate2(xs, xd, A, N, 1))
    return data

# performs simulation 1 (uniform) for range of xs, xd, N_dot
def experiment3(xs_list, xd_list, N_dot_list, N):
    data = []
    for xs in xs_list:
        for xd in xd_list:
            for N_dot in N_dot_list:
                data.append([xs, xd, N_dot], + simulate1(xs, xd, N, N_dot))
    return data

# performs simulation2 (rutherford) for range of xs, xd, A, N_dot
def experiment4(xs_list, xd_list, A_list, N_dot_list, N):
    data = []
    for xs in xs_list:
        for xd in xd_list:
            for A in A_list:
                for N_dot in N_dot_list:
                    data.append([xs, xd, A, N_dot] + simulate2(xs, xd, A, N, N_dot))
    return data

# performs simulation3 (rutherford, modified) for range of xs, xd, A
def experiment5(xs_list, xd_list, A_list, N):
    data = []
    for xs in xs_list:
        for xd in xd_list:
            for A in A_list:
                data.append([xs, xd, A] + simulate3(xs, xd, A, N, 1))
    return data

# performs simulation3 (rutherford, modified) for range of xs, xd, A, N_dot
def experiment6(xs_list, xd_list, A_list, N_dot_list, N):
    data = []
    for xs in xs_list:
        for xd in xd_list:
            for A in A_list:
                for N_dot in N_dot_list:
                    data.append([xs, xd, A, N_dot] + simulate3(xs, xd, A, N, N_dot))
    return data


Simulation Tests

Data Analysis Funcitons

Data Analysis Tests

Measurment Optimization